# QCL(量子回路学習)

* https://pennylane.ai/qml/demos/tutorial_qnn_module_torch.html

<a href="https://colab.research.google.com/github/fuyu-quant/data-science-wiki/blob/main/tabledata/tagledata_regression.ipynb" target="_blank" rel="noopener noreferrer"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%capture
!pip install PennyLane
!pip install torchinfo

In [20]:
import numpy as np
import pandas as pd
import torch
from torchinfo import summary

from sklearn import datasets
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split

import pennylane as qml
from pennylane import numpy as np

## データの用意

In [31]:
iris = datasets.load_iris()
x = iris.data
y = iris.target


In [32]:
x_train, x_valid, y_train, y_valid = train_test_split(x, y, test_size=0.2, random_state=3655)

In [33]:
# 特徴量の標準化
scaler = StandardScaler()
scaler.fit(x_train)

x_train = scaler.transform(x_train)
x_valid = scaler.transform(x_valid)

In [36]:
x_train = torch.tensor(x_train, dtype=torch.float32) #.float()
y_train = torch.tensor(y_train, dtype=torch.int32).long()
#y_train = torch.tensor(int(y_train)).long()
x_valid = torch.tensor(x_valid, dtype=torch.float32)#.float()
y_valid = torch.tensor(y_valid, dtype=torch.int32)#.long()

## NNモデル

In [1]:

layer_1 = torch.nn.Linear(2, 2)
layer_2 = torch.nn.Linear(2, 2)
softmax = torch.nn.Softmax(dim=1)

layers = [layer_1, layer_2, softmax]
model = torch.nn.Sequential(*layers)

/usr/local/lib/python3.9/site-packages/torch/nn/modules/linear.py:96: UserWarning: Failed to initialize NumPy: module compiled against API version 0xf but this version of numpy is 0xe (Triggered internally at /root/pytorch/torch/csrc/utils/tensor_numpy.cpp:77.)
  self.weight = Parameter(torch.empty((out_features, in_features), **factory_kwargs))


## QNNモデル

* 綺麗に描画するためのqiskitのmatplotlibを使う
https://pennylane.ai/blog/2021/05/how-to-visualize-quantum-circuits-in-pennylane/

In [166]:

# pennylaneの通常のシミュレータ
#dev = qml.device("default.qubit", wires=4)

# C++で書かれた高速シミュレータ
dev = qml.device("lightning.qubit", wires=4)
dev = qml.device("qiskit.aer", wires=4)
n_qubits = 4
n_layers = 1

# 状態ベクトルシミュレータ上で時間的にもメモリ的にも効率の良い量子勾配計算法(adjoint)
# https://pennylane.ai/qml/demos/tutorial_adjoint_diff.html
#@qml.qnode(dev, diff_method='adjoint')
@qml.qnode(dev)
def qnode(inputs, weights):
    # デフォルトではRx回転角が使用される
    qml.AngleEmbedding(inputs, wires=range(n_qubits))
    qml.StronglyEntanglingLayers(weights, wires=range(n_qubits))
    #qml.BasicEntanglerLayers(weights, wires=range(n_qubits))
    #qml.AngleEmbedding(inputs, wires=range(n_qubits))
    #qml.StronglyEntanglingLayers(weights, wires=range(n_qubits))
    
    return [qml.expval(qml.PauliZ(wires=i)) for i in range(n_qubits)]
    #return qml.expval(qml.PauliX(0) @ qml.PauliZ(2))


# パラメータの数を取得
weight_shape = qml.StronglyEntanglingLayers.shape(n_layers=4, n_wires=4)

weight_shapes = {"weights": (n_layers, n_qubits, 3)}

# 乱数で重みを作る
weights = np.random.random(weight_shape)

inputs = [0.1, 0.3, 0.4, 0.5]

drawer = qml.draw(qnode, show_all_wires=True)#, wire_order=[2,1,0,3])
print(drawer(inputs, weights))

0: ─╭AngleEmbedding(M0)─╭StronglyEntanglingLayers(M1)─┤  <Z>
1: ─├AngleEmbedding(M0)─├StronglyEntanglingLayers(M1)─┤  <Z>
2: ─├AngleEmbedding(M0)─├StronglyEntanglingLayers(M1)─┤  <Z>
3: ─╰AngleEmbedding(M0)─╰StronglyEntanglingLayers(M1)─┤  <Z>


In [167]:
weight_shapes

{'weights': (1, 4, 3)}

In [168]:
class HybridModel(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.clayer_1 = torch.nn.Linear(4, 4)
        self.qlayer_1 = qml.qnn.TorchLayer(qnode, weight_shapes)
        self.clayer_2 = torch.nn.Linear(4, 3)
        self.softmax = torch.nn.Softmax(dim=1)

    def forward(self, x):
        x = self.clayer_1(x)
        x = self.qlayer_1(x)
        x = self.clayer_2(x)
        return self.softmax(x)

model = HybridModel()

In [169]:
opt = torch.optim.SGD(model.parameters(), lr=0.2)
loss = torch.nn.L1Loss()

In [170]:
summary(model)

Layer (type:depth-idx)                   Param #
HybridModel                              --
├─Linear: 1-1                            20
├─TorchLayer: 1-2                        12
├─Linear: 1-3                            15
├─Softmax: 1-4                           --
Total params: 47
Trainable params: 47
Non-trainable params: 0

## 学習の実行

In [171]:
from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader

train_dataset = TensorDataset(x_train, y_train)
valid_dataset = TensorDataset(x_valid, y_valid)

batch_size = 32
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
valid_dataloader = DataLoader(valid_dataset, batch_size=batch_size, shuffle=False)

In [172]:
#batch_size = 5
batch_size = 32
batches = len(x_train) // batch_size

#data_loader = torch.utils.data.DataLoader(list(zip(X, y_hot)), batch_size=5, shuffle=True, drop_last=True)
import torch.nn as nn

criterion = nn.CrossEntropyLoss()


epochs = 20

for epoch in range(epochs):

    running_loss = 0

    for xs, ys in train_dataloader:
        opt.zero_grad()
        #print(ys)
        outputs = model(xs)
        #print(outputs)
        #y_valid = torch.tensor(y_valid, dtype=torch.int32)

        loss_evaluated = criterion(outputs, ys)
        loss_evaluated.backward()

        opt.step()

        running_loss += loss_evaluated

    avg_loss = running_loss / batches
    print("Average loss over epoch {}: {:.4f}".format(epoch + 1, avg_loss))

y_pred = model(x_valid)
predictions = torch.argmax(y_pred, axis=1).detach().numpy()

correct = [1 if p == p_true else 0 for p, p_true in zip(predictions, y_valid)]
accuracy = sum(correct) / len(correct)
print(f"Accuracy: {accuracy * 100}%")


Average loss over epoch 1: 1.4665
Average loss over epoch 2: 1.4596
Average loss over epoch 3: 1.4482
Average loss over epoch 4: 1.4345
Average loss over epoch 5: 1.4154
Average loss over epoch 6: 1.3913
Average loss over epoch 7: 1.3565
Average loss over epoch 8: 1.3197
Average loss over epoch 9: 1.2816
Average loss over epoch 10: 1.2494
Average loss over epoch 11: 1.2192
Average loss over epoch 12: 1.1946
Average loss over epoch 13: 1.1723
Average loss over epoch 14: 1.1551
Average loss over epoch 15: 1.1435
Average loss over epoch 16: 1.1232
Average loss over epoch 17: 1.1066
Average loss over epoch 18: 1.0954
Average loss over epoch 19: 1.0813
Average loss over epoch 20: 1.0631
Accuracy: 86.66666666666667%
